# Guide RNA design  

In [91]:
from Bio import Seq
import re
from itertools import product 
from ipywidgets import *  
from IPython.display import display

In [92]:
exampleSequence = "TGAGAGAAGAGTTTCATATTTGCAAGGTCTCAGACATGCCTTTAAAATTTCATAATACTTCTTCTTGTGTTTCCCATGCTTGATGGGGCTCAGAATTGACAGTGTACATTTTCAGTAATACAGAGATGAGAAGGGTCAGAGAGGAAGTAAAGTGGCTAGGACTGGCTATGTAAATTGCCAAGGGGTGCAGTTCAAATGAAAATGTGAGACCTCTAATATAAAATTTCATGATGGGGATAATGGATCATGAAACTAATCGTGGGGCTTTGTGCGACACCACGGGACACTTGCCTACAAAGCCATCCCTACTCCCTGCTTAAACGGAGACAAGAACACATTGGTGATTCTCAAATGCAAACTGTCCCTTTACTGAAAACCTAATGAGTTTAATGCTTACTATACTGTAGTCCCTTATGTGCTGATTACATTGTGGAATGCTGCAGGGAGAAAAACAAATTCACCTAATGATGCTAAAGAACCATTAGGAGACTTTACTAGTTTAGGTCACAAGTGCCTAGAAATACAGAAATGATCTTTGACCTTTCCTCCATTTTTCAAAAGGACGCTATTTCTGTGAACCTCATTGACCTCTCCAGACAAAGTCCCAGATTGCCTTTGCCAAAGTTAACAACCCATCTTGGCTTCTTGCCATTTCTCATGTATTCACTAGATCAGTGCATGACTGCTGCTCTCAACCCACATCATGGGGCCAGAGCCTTCACTGGTTAATATTTATAAAAAAATACTTTGAAGATTAAATCCTTGGTCAGTAGAGAAAACACTAGACATGGATAGAACAAGAAATTGTGGGGTCTGGGCCTCCTCCCAGAACTGCCACCACCAGACAATGTTATCTTTGACAGATTTGTGGTATCTGGGTGGCTGTACTTTTCTTTTGGTGAAATAGCAAAAGCCAAAAAAAGAGACTGTAACATCTGATGGGATTTTGCCCAGCTCCAGTACTGAATTCCTGGGTTGCCTTAGAGGAAAGGTATTGACTGTTTCATCTCTTTGGCCCTCAGTATACTCCCCCAACCCAAATCCACCCCTCCGTCATACACACATCTTACTCTGGAAATGATTCTTAGCAGGAAAACATACTATTATGAGGAACATACTTAGCAGTGCTCTCATAACCCTGCCAAGGGCTGGCATTTTGATTTTGTTTGCACACAGAAACCTGTTTGACTGAAATCTGGAAAAAAAAAAAAAAAAAAAAGAAAAACCAGACACGGGAGAAAAATAAAAATCACAGTAGTTTTATACATACATTTGATTTCTAAGGCCAGATAACTGATTTTGTTACCATTTTCTATCCTGTTATTCTGGTGCTGCGAATGCACTAGCTAGAAGTAAAAGTTGTGGGGAAAGGAAGAAGTTGGTTCTGAGTTTATTTTCCCCAGGGTTTCTGAGGGAGTTCCACCCTCTCCCCTGAAAGGGAGTTGGGGCAAATCTATGTTTCTAATTCCACATCAAGTGTAACTATAGCCTGTTCAGAATATTCCTCTTTGAAACAGCACTTTGAGAACAGGGAGCTGCAGCAGTGCAAGACAGCTTGTGGTTGGCTTGGGACTTTAGCCTCAACCCAAATTTGGAACTGGGAACATCTCCAGTCAGCTCTATGGCTAGTCTCACCTTCCTGATCAGCCACTTGGGCGGGCTGTAAGCCTGGCTAAACCAGGTCACCTCAACATTTGTGTCAGGGCAAAGAGTGGAGGAGAGAAGGTGTTTCTTCAACTCCTGGCTTTTAGCTGTGCCTGGATGCCTGCATACTGTTAAAATTCAGCTGCCAGCCTTATCTTATCCGTAATCAGTTGAATTTATTACAATAATCTGATTTTAGAAGCCCCTGAGGCATGGCTTGAGGTCCTATATTGACTTAGGCCCTTGGCTCTGAGGCATTAGAGTCCTGAATTGACTTTAAAGAGGCATAGAAACATAGATGTCTACCAAAGGCTTCAAACTGGCCAACTCTCAGTCTATTTCAACATGCCCTACTGAGCCAATCACTCTGTGCCAAGCCAGTACTAAGAACTGGGGGGACAAAGTGGACTGAGTCTACTCCCTGCTCTTCAGAAGCTCAAAGGCTGTTTATGAAACCAGAGAAGTAAATATATGCTTTCAATATAATGTGATTAATTATAGGCTAGAAATGAGATTGAATGCCTTGAAAGCACTGAGGGTGAGGATCCAACACAACCTAGGGGCATCGGGGAAGACTTCCTGTAAAAGGGATCGCCAGGTAGGGTTGAAAGTAGTTGGGGGGAAACAGTGAGAAATAAATGCCAAAGAAGAAGCAGAAAAAGCAAAGGCAAGGAGGTCTCTTACAGCACTCTATGCTCAGGAAATTATAATGAGTCAGGATAAGTAAACCT"

In [93]:
# funcion to extend ambiguous IUPAC letters. This function generates all possible sequences, 
# expanding the characters defined by the IUPAC standard, expect 'N' (since that would create a huge overhead)
def extend_ambiguous_expectN_dna(seq):
    """return list of all possible sequences given an ambiguous DNA input"""
    d = Seq.IUPAC.IUPACData.ambiguous_dna_values
    d['N'] = 'N' # here we exclude expansion of N
    r = []
    for i in product(*[d[j] for j in seq]):
        r.append("".join(i))
    return r 

In [94]:
def on_button_clicked(b):
    cleanMotif = re.sub(r"[^ACTGRYSWKMBDHVN]", '',motif.value.upper())
    cleanMotif_rc = str(Seq.Seq(cleanMotif,Seq.IUPAC.ambiguous_dna).reverse_complement())
    # the IUPAC letters are allowed, but not matched in Sequence
    cleanSequence = re.sub(r"[^ACTGRYSWKMBDHV]", '',sequence.value.upper())
    #print("Cleaned motif:",cleanMotif,"\n")
    #print("Cleaned sequence:\n",cleanSequence)
    #print("\nMatches found:\n")
    res = ''
    res = res + 'browser position '+str(chrom.value)+':'+str(chrPos.value) +'-' + str(chrPos.value+len(cleanSequence))+'\n'
    res = res + 'browser hide all\n'
    res = res + 'track name="ColorByStrandDemo" description="Color by strand demonstration" '+ 'visibility=2 colorByStrand="255,0,0 0,0,255"\n'

    # forward
    i = 1
    for iupacMotif in extend_ambiguous_expectN_dna(cleanMotif):
        regexMotif = re.sub(r"N", '.',iupacMotif)
        #for match in re.finditer(regexMotif, cleanSequence): # finds only non-overlapping matches
        for match in re.finditer(r'(?=('+regexMotif+'))', cleanSequence):
            mm = match.group(1)
            # we need only certain part of the matched sequence, defined by outStart and outLength (0 based indexing)
            idx1 = outStart.value - 1
            idx2 = idx1 + outLength.value
            #print(match.start()+idx1,mm[idx1:idx2], mm)
            res = res +  chrom.value + '\t' + str(chrPos.value+match.start()+idx1) + '\t' + str(chrPos.value+match.start()+idx2-1) + '\t' + 'gRNA_' + str(i) + '_' + str(mm[idx1:idx2]) + '\t0\t+\n'
            #res = res + str(match.start()+idx1+1) +' ' + str(mm[idx1:idx2]) + '+\n'
            i = i+1

    # reverse complement
    i = 1
    for iupacMotif_rc in extend_ambiguous_expectN_dna(cleanMotif_rc):
        regexMotif_rc =  re.sub(r"N", '.',iupacMotif_rc)
        #for match in re.finditer(regexMotif_rc, cleanSequence): # finds only non-overlapping matches
        for match in re.finditer(r'(?=('+regexMotif_rc+'))', cleanSequence):
            mm = match.group(1)
            # we need only certain part of the matched sequence, defined by outStart and outLength (0 based indexing)
            idx2 = len(regexMotif_rc) - (outStart.value - 1)
            idx1 = len(regexMotif_rc) - (outStart.value - 1 + outLength.value)
            #print(match.start()+idx1,mm[idx1:idx2], mm)
            res = res +  chrom.value + '\t' + str(chrPos.value+match.start()+idx1) + '\t' + str(chrPos.value+match.start()+idx2-1) + '\t' + 'gRNArc_' + str(i) + '_' + str(mm[idx1:idx2]) + '\t0\t-\n'
            #res = res + str(match.start()+idx1+1) +' ' + str(mm[idx1:idx2]) + '-\n'
            i = i+1

    resultBox.value = res
    resultBox.update_config
    
def handle_motif(m):
    return


In [96]:
tBox_layout = Layout(height='400px',width='800px')
smallBox_layout = Layout(width='200px')
miniBox_layout = Layout(width='40px')

motif = widgets.Text(description='Motif:', value='GNNNNNNNNNNNNNNNNNNNNGG',width='50%')
motif.on_submit(handle_motif)
#TODO: use: max=len(motif.value), but it is not updated when motif is changed 
outStart = widgets.BoundedIntText(description='From:', min=1, max=999999, value=1,width='13%')
outLength = widgets.BoundedIntText(description='Length:', min=1, max=999999, value=20,width='13%')
chrom = widgets.Text(description='Chr:', value='chrX',width='20%')
chrPos = widgets.BoundedIntText(description='ChrStart:', min=1, max=999999999, value=67523514,width='20%')

startButton = widgets.Button(description='Start',  button_style='info')
startButton.on_click(on_button_clicked) 
#startButton.background_color = "cornflowerblue"
#startButton.background_color = 'blue'
#startButton.button_style = 'Info'
#startButton.font_weight = 'bold'
#startButton.font_size = '14pt'

page0 = widgets.Box(children=[widgets.HBox(children=[motif,outStart,outLength]),widgets.HBox(children=[chrom,chrPos])])
#chrom,chrPos
accord0 = widgets.Accordion(children=[page0], width=800, height=400)
accord0.set_title(0, 'Motif')

sequence = widgets.Textarea(description='', width='90%',height='500px', value=exampleSequence)
#zip1 = widgets.BoundedIntText(description='Zip:', min=0, max=99999)
#page1 = widgets.Box(children=[name1, zip1])
page1 = widgets.Box(children=[sequence])
accord1 = widgets.Accordion(children=[page1], width=800, height=400)
accord1.set_title(0, 'Sequence')

resultBox = widgets.Textarea(description='', width='90%',height='500px', value='')
page2 = widgets.Box(children=[resultBox])
accord2 = widgets.Accordion(children=[page2], width=800, height=400)
accord2.set_title(0, 'Results')

display(startButton,accord0,accord1,accord2)



This tool takes a genetic _Sequence_ and looks for patterns which match the given _Motif_. In the motif 'N' character matches any letter in the sequence. User can truncate the displayed matching sequences by setting the _From_ and _Length_ fields.   
After pressing the _Start_ button, on the output the truncated matcing sequences, and their positions are listed.  

- IUPAC characters, other than ACGT are handled in Motif, but not in Sequence
- all lower case characters in the sequence are converted to upper case
- all characters except 'A','C','G','T' (including whitespaces and linebreake) will be removed from Sequence
- all characters except 'A','C','G','T' and IUPAC characters: RYSWKMBDHV, and 'N' - which matches any base will be removed from Motif
- !!! output BED files use 0-based indexing!!! See: https://genome.ucsc.edu/FAQ/FAQformat.html#format1
